In [ ]:
import requests
import time
import csv
import matplotlib
import matplotlib.pyplot as plt

In [1]:
BANK_SERVICE_URL = "https://latest-960957615762.me-central1.run.app"
BALANCE_ENDPOINT = f"{BANK_SERVICE_URL}/getbalance"
LOGS_ENDPOINT = f"{BANK_SERVICE_URL}/getlogs"
NUMBER_OF_REQUESTS = 100
REQUEST_TIMEOUT = 10
DELAY_BETWEEN_REQUESTS = 0.1

In [ ]:
def invoke(): # invoking the banking service 100 times to generate log data
    print(f"[1/3] Invoking {NUMBER_OF_REQUESTS} times")
    cnt = 0
    
    for i in range(NUMBER_OF_REQUESTS):
        try:
            response = requests.get(BALANCE_ENDPOINT, timeout=REQUEST_TIMEOUT)
            print(f"Request № {i+1} out of {NUMBER_OF_REQUESTS}: Status {response.status_code}")
        except requests.exceptions.RequestException as e:
            print(f"Request № {i+1} out of {NUMBER_OF_REQUESTS}: ERROR: ({type(e).__name__})")
        
        cnt += 1
        time.sleep(DELAY_BETWEEN_REQUESTS)

    print(f"{cnt} Requests are DONE!")